# Borrower reliability study

The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect repaying the loan in time. Input data from the bank - statistics on the solvency of customers.

The results of the study will be taken into account when building a **credit scoring** model - a system that evaluates the ability of a potential borrower to repay a loan to a bank.

## 1. Examine the general information

In [1]:
import pandas as pd 
from pymystem3 import Mystem

In [2]:
data = pd.read_csv('/datasets/data.csv')  
display(data.head(10)) 
data.info() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Conclusions**

The table has a number of data problems:

1. There is missing data on the number of days of service and income - the same number of gaps in both columns. This may indicate either that the person is not working yet and does not have income, or that the data has not been filled in deliberately or has been lost. <br>
2. The number of days of service has negative and illogical values - in index 4, the number of days is more than 340,000, which is more than 900 years. This is impossible and makes no sense. <br>
3. There is a different register in the education column . It is not clear yet whether this will be a problem for the further analysis, yet, the register will be converted to a single lower case.<br>
4. Columns with income and number of days of service have a float data type, it is not yet clear whether this will be a problem for the further analysis, but it would be reasonable to convert the data to an int type.

## 2. Data preprocessing

### 2.1 Missing values processing

In [10]:
data[data['days_employed'].isna()]['total_income'].unique() 

array([nan])

The gaps in the data on the number of days of service and income have already been described above. At this step, we made sure that the gaps match not only in quantity, but are also present in the same lines.

The gaps are non-random, because they are absent in the same lines and may be due to the fact that the Clients deliberately did not indicate the data, or the data was lost during uploading.

Since the data in both columns is quantitative, it makes sense to replace them with a representative variable. It would be more correct to replace the income with the median in order not to distort the sample data. In addition, the data will be needed for further analysis. Years of experience can be replaced by the average - years of experience do not affect further analysis.

In [11]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median()) 

In [12]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean()) 
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### 2.2 Data type replacement

In [13]:
data['total_income'] = data['total_income'].astype('int') # замена типа float на int по колонке дохода 
data['days_employed'] = data['days_employed'].astype('int') # замена типа float на int по колонке годы стажа
display(data.dtypes['total_income']) # проверка замены
data.dtypes['days_employed'] # проверка замены

dtype('int64')

dtype('int64')

### 2.3 Duplicates processing

In [14]:
data['education'].sort_values().unique() #unique values Education

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [15]:
data['education'] = data['education'].str.lower() #lower case
data['education'].sort_values().unique() #check

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

Также предлагается проверить , присутствуют ли в таблице полные дубликаты.

In [16]:
data.duplicated().sum() #check for full duplicates

71

Since the table does not contain any unique user identifier (phone number, email, ID), it can be assumed that 71 out of 21,525 people (0.3%) can have a complete match of data. Consequently, it is proposed not to delete complete duplicates.

### 2.4 Lemmatization

With the help of Pymystem library, lemmas were derived for all values of the `purpose` column for the further use in the study.

In [17]:
m = Mystem() 

for i in range(data.shape[0]):
    lemmas = m.lemmatize(data['purpose'][i]) #find lemma for each value in Purpose
    data.loc[i, 'lemmas'] = ','.join(lemmas) 

display(data.head()) #check



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"покупка, ,жилье,\n"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"приобретение, ,автомобиль,\n"
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"покупка, ,жилье,\n"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"дополнительный, ,образование,\n"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"сыграть, ,свадьба,\n"


### 2.5 Data categorization

In order to analyze data for the scoring system - it is proposed to split it into several categories:

1. Children-wise
2. Income-wise
3. Purpose-wise

**1 category** <br>

In [18]:
data['children'].value_counts() #unique values with its count

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Values for "-1" and "20" indicate a potential error in the data. Perhaps there was an error during data transfer, 2.0 turned into 20, and 1 into -1. It is proposed to replace the values with 1 and 2, respectively.

In [19]:
data['children'] = data['children'].replace(-1,1) 
data['children'] = data['children'].replace(20,2) 
data['children'].value_counts() 

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

There can be 3 categories:
1. Clients without children
2. Clients with one child
3. Clients with more than one child

In [20]:
def children_id(children): 
    if children == 0:      
        return 'без детей' 
    elif children == 1:    
        return '1 ребенок' 
    else:
        return 'более 1 ребенка' 
    
data['children_id'] = data['children'].apply(children_id) 

data.head() #check

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,children_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"покупка, ,жилье,\n",1 ребенок
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"приобретение, ,автомобиль,\n",1 ребенок
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"покупка, ,жилье,\n",без детей
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"дополнительный, ,образование,\n",более 1 ребенка
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"сыграть, ,свадьба,\n",без детей


**2 category** <br>

In [21]:
print('Минимальный доход:', data['total_income'].min()) #Minimal income
print('Максимальный доход:', data['total_income'].max()) #Max income
print('Среднее:', data['total_income'].mean()) #Average income
print('Медиана:', data['total_income'].median()) #Median income

Минимальный доход: 20667
Максимальный доход: 2265604
Среднее: 165158.94276422763
Медиана: 145017.0


There are 4 categories: low income, below average, above average, high income. <br>
Let low income be: from min to average/2 (inclusive) <br>
Below Average: Average/2 to Average (inclusive) <br>
Above Average: Intermediate to Intermediate * 1.5 (inclusive) <br>
High income: from average * 1.5 to max.

In [22]:
def total_income_id(total_income): 
    if total_income <= 165159/2 : 
        return 'низкий доход' 
    elif 165159/2 < total_income <= data['total_income'].mean() : 
        return 'ниже среднего' 
    elif data['total_income'].mean() < total_income <= 165159*1.5 : 
        return 'выше среднего' 
    else:
        return 'высокий доход' 
    
data['total_income_id'] = data['total_income'].apply(total_income_id) 
data.head() #check

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,children_id,total_income_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"покупка, ,жилье,\n",1 ребенок,высокий доход
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"приобретение, ,автомобиль,\n",1 ребенок,ниже среднего
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"покупка, ,жилье,\n",без детей,ниже среднего
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"дополнительный, ,образование,\n",более 1 ребенка,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"сыграть, ,свадьба,\n",без детей,ниже среднего


**3 category** <br>

In [23]:
data['purpose'].unique() 

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

The main categories that can be distinguished are: <br>
1. buying a car <br>
2. purchase / construction of a house / real estate <br>
3. getting an education <br>
4. wedding <br>
5. repair

In [24]:
def purpose_id(purpose_lemma): 
    if 'автомобиль' in purpose_lemma: 
        return 'авто' 
    elif 'ремонт' in purpose_lemma: 
        return 'ремонт' 
    elif 'образование' in purpose_lemma: 
        return 'образование'         
    elif 'свадьба' in purpose_lemma: 
        return 'свадьба' 
    elif 'недвижимость' or 'жилье' in purpose_lemma: 
        return 'жилье' 
    else:
        return 'категория не задана' # if some lemmas are not included above - return "no category" 
    
data['purpose_id'] = data['lemmas'].apply(purpose_id) 
print(data['purpose_id'].unique()) #check
data.head() #check

['жилье' 'авто' 'образование' 'свадьба' 'ремонт']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,children_id,total_income_id,purpose_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"покупка, ,жилье,\n",1 ребенок,высокий доход,жилье
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"приобретение, ,автомобиль,\n",1 ребенок,ниже среднего,авто
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"покупка, ,жилье,\n",без детей,ниже среднего,жилье
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"дополнительный, ,образование,\n",более 1 ребенка,высокий доход,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"сыграть, ,свадьба,\n",без детей,ниже среднего,свадьба


**Вывод**

There were 3 categories in total:
1) by the number of children <br>
2) by income level<br>
3) according to the purpose of the loan<br>

Categories were selected based on the objectives of the study. Additional categorization by marital status was not carried out, as the column already contains the necessary categories.

## 3. Research questions

- Is there a relationship between having children and repaying a loan on time?

In [25]:
data.groupby('children_id')['debt'].value_counts(normalize = True) 

children_id      debt
1 ребенок        0       0.908530
                 1       0.091470
без детей        0       0.924871
                 1       0.075129
более 1 ребенка  0       0.907208
                 1       0.092792
Name: debt, dtype: float64

<u>**Conclusion 1**</u>

The clients with children have the loan default rate of 9.3% (for clients with more than 1 child) and 9.1% (for clients with one child). On average (9.2%), these values are 1.7 percentage point higher than the loan default rate for clients without children (7.5%).

**Thus, the number of children has a slight negative impact on the repayment of the loan on time.**

- Is there a relationship between marital status and loan repayment on time?

In [26]:
data_family_pivot = data.pivot_table(index = 'family_status', values = 'debt', aggfunc= ['sum','count']) 
data_family_pivot['percentage_non_repayment'] = data_family_pivot['sum']/data_family_pivot['count'] 
data_family_pivot.sort_values(by = 'percentage_non_repayment') 

,sum,count,percentage_non_repayment
,debt,debt,
family_status,,,
вдовец / вдова,63,960,0.065625
в разводе,85,1195,0.071130
женат / замужем,931,12380,0.075202
гражданский брак,388,4177,0.092890
Не женат / не замужем,274,2813,0.097405


<u>**Conclusion 2**</u>

The more "serious" the client's marital status, the lower the loan default rate. The difference between the category of clients `single` and `in a civil marriage` was only 0.4 p.p. (9.7% and 9.3%), while clients from the category `married` (7.5%) have 1.8 p.p. higher chance of repaying the loan on time. 

The category `divorced` most likely indicates that the client has been married for some time, probably even for a long time, as does the category of `widower`. The percentage of non-return falls proportionally - in the divorced category it is already 7.1%, in the category of widowers - 6.6%, which is 0.4 p.p. and 1 p.p., respectively, below the married category. However, it is important to understand that the percentage of non-return in the "widower" category may also be age factor related.

**Thus, the presence of a marriage in the present or in the past suggests that the client is more reliable.**

- Is there a relationship between income level and loan repayment on time?

In [27]:
data_income_pivot = data.pivot_table(index = 'total_income_id', values = 'debt', aggfunc= ['sum','count']) 
data_income_pivot['percentage_non_repayment'] = data_income_pivot['sum']/data_income_pivot['count'] 
data_income_pivot.sort_values(by = 'percentage_non_repayment') 

,sum,count,percentage_non_repayment
,debt,debt,
total_income_id,,,
высокий доход,198,2883,0.068678
низкий доход,196,2547,0.076953
выше среднего,399,4808,0.082987
ниже среднего,948,11287,0.083990


<u>**Conclusion 3** </u>

As expected - the lowest non-return rate was for the high-income category - 6.87% of loan non-return. At the same time, it was interesting that clients from the low-income category are in the second place - 7.7% of non-return. People with an average income make up the most "unreliable" group and repay the loan late by about 0.6 p.p. more often: 8.3% - clients with an income above the average and 8.4% - clients with an income below the average (the most massive category).

The categorization can be improved by adding the ranking of income used by the bank.

- How do different loan purposes affect its repayment on time?

In [28]:
data_purpose_pivot = data.pivot_table(index = 'purpose_id', values = 'debt', aggfunc= ['sum','count']) 
data_purpose_pivot['percentage_non_repayment'] = data_purpose_pivot['sum']/data_purpose_pivot['count'] 
data_purpose_pivot.sort_values(by = 'percentage_non_repayment') 

,sum,count,percentage_non_repayment
,debt,debt,
purpose_id,,,
ремонт,35,612,0.057190
жилье,747,10228,0.073035
свадьба,186,2348,0.079216
образование,370,4022,0.091994
авто,403,4315,0.093395


<u>**Conclusion 4** </u>

The worst situation with loan repayment on time is in the category of auto loans - almost every 10th loan will be repaid late (9.34%). This may be due to the fact that the car is usually pledged to the bank for such loans and customers feel a little more relaxed about loan repayment. The slightly higher percentage of non-return is in the category of education - 9.2% - may be due to the fact that the income of clients falls when they return to education, or is absent (higher education). 

Slightly better is the situation with loans for housing and weddings - 7.9% and 7.3% of loan defaults, respectively. The most "repaid" on time loan is a loan for repairs - 5.7% default, which may be due to the small loan amounts. 

However, it is also the smallest category.

## 4. Final conclusion

Repayment of the loan on time depends on various factors, including: the presence of children, marital status, income level, purpose of the loan, and others not specified in this study.

It was found out that the presence of children and marital status affect the repayment of the loan on time. At the same time, there is some contradiction: if the client is with children, the chance that the loan will be returned on time is less; at the same time, married clients have a higher chance of repaying the loan on time than single clients.

The level of income also has an effect. People with average income are less likely to repay the loan on time than people with high and even low incomes (the result may be distorted by the fact that average category is the most massive - there is class disbalance).

As for the purpose of the loan, the range of values is most noticeable. The lowest percentage of loan repayment on time is in the car category and the education category, while repairs are returned mostly on time. Loans for housing and for a wedding are not returned on time with approximately the same frequency.